In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

# Chrome 브라우저를 사용하여 웹 드라이버 시작
driver = webdriver.Chrome()

# Google 홈페이지로 이동
driver.get("https://www.google.com/")

# 페이지가 로딩될 때까지 잠시 기다립니다.
time.sleep(2)

# Google 검색 입력란을 찾고 "Selenium"이라는 검색어를 입력합니다.
search_box = driver.find_element(By.NAME, "q")
search_box.send_keys("Selenium")
search_box.send_keys(Keys.RETURN)

# 검색 결과가 로딩될 때까지 잠시 기다립니다.
time.sleep(2)

# 검색 결과에서 첫 번째 결과의 제목을 가져옵니다.
first_result = driver.find_element(By.CSS_SELECTOR, "div.g h3")
print("첫 번째 검색 결과: ", first_result.text)

# 웹 브라우저를 닫습니다.
driver.quit()

첫 번째 검색 결과:  Selenium


In [ ]:
baseURL = 'https://www.agoda.com/?cid=1719676'